In [1]:
import pandas as pd
from config import CONFIGS
import os
import re
import os
import json
import gc
from typing import Tuple

from utils.processing_functions import load_file_local_first, save_file_local_first, explode_columnar_df
from utils.weaviate_client import WeaviateClient

from sklearn.preprocessing import MinMaxScaler


ENVIRONMENT = os.environ.get("ENVIRONMENT", "dev")
S3_SCRAPER_BUCKET = CONFIGS["s3_scraper_bucket"]
GAME_CONFIGS = CONFIGS["games"]
RATINGS_CONFIGS = CONFIGS["ratings"]
IS_LOCAL = True if os.environ.get("IS_LOCAL", "False").lower() == "true" else False

read_dir = "data/prod/games/game_dfs_clean/"

weaviate_client = WeaviateClient()
weaviate_client.connect_weaviate_client_docker()


IS_LOCAL: True

Checking for local config file and evaluating for updates from S3.
Loading config from local


/Users/jennywadkins/.local/share/virtualenvs/boardgamegeek-ZH0FNRKg/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:504: UserWarning: <module 'weaviate.client' from '/Users/jennywadkins/.local/share/virtualenvs/boardgamegeek-ZH0FNRKg/lib/python3.12/site-packages/weaviate/client.py'> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


# Content Similarity Models

In [26]:
name_df = pd.read_pickle(f"{read_dir}games_clean.pkl")
name_df = name_df.sort_values("BayesAvgRating", ascending=False)[
            0 : 1000
        ].reset_index(drop=True)
name_df = name_df[['BGGId','Name']]

df_cleaned_rag_sentences = pd.read_pickle('top_1000_cleaned_rag.pkl')
df_cleaned_rag_sentences = df_cleaned_rag_sentences[['BGGId','Description','About']]
df_cleaned_rag_sentences = name_df.merge(df_cleaned_rag_sentences, on="BGGId", how="inner")

df = pd.read_pickle("top_1000_cleaned_rag_with_ratings_extrap.pkl")
df = df.drop(columns=['Description','About',"Positive_Components","Negative_Components","Positive_Sentences","Negative_Sentences"])
df = df_cleaned_rag_sentences.merge(df, on="BGGId", how="inner")
df = df.rename(columns={x:x.lower().replace(" ","_") for x in df.columns.tolist()})
df = df.fillna(0)

df.head(3)

,bggid,name,description,about,positive_strategic_depth,positive_excellent_design,positive_player_interaction,positive_replayability,positive_thematic_integration,positive_component_quality,...,negative_anticlimactic_endings,positive_excellent_production_values,positive_attractive_artwork,positive_accessible_non_gamers,positive_variable_experiences,negative_catch_up_mechanics,positive_simple_to_learning,negative_action_limitations,positive_good_themes,positive_good_interactions
0,224517,Brass: Birmingham,brass: birmingham is an economic strategy game...,this game is a strategic economic simulation s...,1.000,1.000,1.000,1.000,1.000,1.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,161936,Pandemic Legacy: Season 1,pandemic legacy is a cooperative campaign game...,this game is a cooperative board game where pl...,0.834,0.739,0.848,0.951,0.746,0.683,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,174430,Gloomhaven,gloomhavenis a game of euroinspired tactical c...,this game is a cooperative tactical adventure ...,1.000,0.853,0.785,1.000,0.610,0.893,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
test_games = ["Dominion", "Gloomhaven", "Pandemic", "Splendor", "Viticulture Essential Edition", "Great Western Trail", "Terraforming Mars", "Chess", "Azul", "Codenames", "Pandemic Legacy: Season 1"]

compare_df_storage = {}

game_name = test_games[1]

positive_columns = [x for x in df.columns if x.startswith('positive_')]
negative_columns = [x for x in df.columns if x.startswith('negative_')]

all_columns_to_produce = positive_columns + negative_columns

### Mechanics Only

In [37]:
mech_df = pd.read_pickle(f"{read_dir}mechanics_clean.pkl")
mech_df = explode_columnar_df(mech_df)


mech_df.head()

mechanic,Acting,Action / Dexterity,Action / Event,Action Points,Action Queue,Action Retrieval,Action Timer,Advantage Token,Alliances,Area Majority / Influence,...,Traitor Game,Trick-taking,Tug of War,Variable Phase Order,Variable Player Powers,Variable Set-up,Victory Points as a Resource,Voting,Worker Placement,Zone of Control
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [40]:
mechanics_attributes = mech_df.columns.to_list()
mechanics_attributes

['Acting',
 'Action / Dexterity',
 'Action / Event',
 'Action Points',
 'Action Queue',
 'Action Retrieval',
 'Action Timer',
 'Advantage Token',
 'Alliances',
 'Area Majority / Influence',
 'Area Movement',
 'Area-Impulse',
 'Auction or Bidding',
 'Automatic Resource Growth',
 'Betting and Bluffing',
 'Bias',
 'Bids As Wagers',
 'Bingo',
 'Bribery',
 'Campaign / Battle Card Driven',
 'Card Play Conflict Resolution',
 'Catch the Leader',
 'Chaining',
 'Chit-Pull System',
 'Command Cards',
 'Commodity Speculation',
 'Communication Limits',
 'Connections',
 'Constrained Bidding',
 'Contracts',
 'Cooperative Game',
 'Crayon Rail System',
 'Critical Hits and Failures',
 'Cube Tower',
 'Deck Construction',
 'Deck, Bag, and Pool Building',
 'Deduction',
 'Delayed Purchase',
 'Dice Rolling',
 'Die Icon Resolution',
 'Different Dice Movement',
 'Drafting',
 'Drawing',
 'Elapsed Real Time Ending',
 'Enclosure',
 'End Game Bonuses',
 'Events',
 'Exploration',
 'Finale Ending',
 'Flicking',
 'Fol

In [36]:
collection_name = "mechanics_only"

mech_df = pd.read_pickle(f"{read_dir}mechanics_clean.pkl")
mech_df = explode_columnar_df(mech_df).reset_index()
mechanics_only = name_df.merge(mech_df, on="BGGId", how="left")
del mech_df
gc.collect()

weaviate_client.create_bgg_collection(collection_name=collection_name, reset=False)

mechanics_only = weaviate_client.add_bgg_collection_batch(df=mechanics_only, collection_name=collection_name)

mechanics_only.to_pickle('mechanics_only.pkl')

KeyError: 'bggid'

### Pros/Cons Keywords Only

In [4]:
# collection_name = "keywords_only"

# keyword_df = df.copy().drop(columns=['about','description'])

# weaviate_client.create_bgg_collection(collection_name=collection_name, reset=False, attributes=all_columns_to_produce)

# keyword_df = weaviate_client.add_bgg_collection_batch(df=keyword_df, collection_name=collection_name, attributes=all_columns_to_produce)

# keyword_df.to_pickle('keywords_only_df.pkl')

In [5]:
collection_name = "keywords_only"

keyword_df = pd.read_pickle('keywords_only_df.pkl')
compare_df_storage[collection_name] = keyword_df

uuid = keyword_df.loc[keyword_df['name']==game_name]['UUID'].values[0]

similars = weaviate_client.find_near_objects(collection_name=collection_name, uuid=uuid, limit=50)

picks = {}

for item in similars:
    picks[str(item.uuid)]=item.metadata.distance
    
picks = pd.DataFrame.from_dict(picks, columns=['distance'], orient='index').sort_values(by='distance', ascending=True).reset_index().rename(columns={'index':'UUID'})

picks = picks.merge(keyword_df, on='UUID', how='inner')[1:]
picks[['name','distance']].head(5)

,name,distance
1,Frosthaven,0.250484
2,Twilight Struggle,0.340128
3,Twilight Inscription,0.353865
4,Gloomhaven: Jaws of the Lion,0.358690
5,Cloudspire,0.362963


### About Only

In [6]:
# collection_name = "about_only"

# about_df = df[['bggid',	'name',	'about']]

# weaviate_client.create_bgg_collection(collection_name=collection_name, reset=False, use_about=True)

# about_df = weaviate_client.add_bgg_collection_batch(df=about_df, collection_name=collection_name, use_about=True)

# about_df.to_pickle('about_df.pkl')

In [7]:
collection_name = "about_only"

about_df = pd.read_pickle('about_df.pkl')
compare_df_storage[collection_name] = about_df

uuid = about_df.loc[about_df['name']==game_name]['UUID'].values[0]

similars = weaviate_client.find_near_objects(collection_name=collection_name, uuid=uuid, limit=50)

picks = {}

for item in similars:
    picks[str(item.uuid)]=item.metadata.distance
    
picks = pd.DataFrame.from_dict(picks, columns=['distance'], orient='index').sort_values(by='distance', ascending=True).reset_index().rename(columns={'index':'UUID'})

picks = picks.merge(about_df, on='UUID', how='inner')[1:]
picks[['name','distance']].head(5)

,name,distance
1,Frosthaven,0.159823
2,Gloomhaven: Jaws of the Lion,0.173312
3,Shadows of Brimstone: Swamps of Death,0.187943
4,The 7th Citadel,0.197546
5,Descent: Legends of the Dark,0.199045


### About and Description

In [8]:
# collection_name = "about_and_desc"

# desc_df = df[['bggid','name','about', 'description']]

# weaviate_client.create_bgg_collection(collection_name=collection_name, reset=False, use_about=True, use_description=True)

# about_df = weaviate_client.add_bgg_collection_batch(df=desc_df, collection_name=collection_name, use_about=True, use_description=True)

# desc_df.to_pickle('desc_df.pkl')

In [9]:
collection_name = "about_and_desc"

desc_df = pd.read_pickle('desc_df.pkl')
compare_df_storage[collection_name] = desc_df

uuid = desc_df.loc[desc_df['name']==game_name]['UUID'].values[0]

similars = weaviate_client.find_near_objects(collection_name=collection_name, uuid=uuid, limit=50)

picks = {}

for item in similars:
    picks[str(item.uuid)]=item.metadata.distance
    
picks = pd.DataFrame.from_dict(picks, columns=['distance'], orient='index').sort_values(by='distance', ascending=True).reset_index().rename(columns={'index':'UUID'})

picks = picks.merge(desc_df, on='UUID', how='inner')[1:]
picks[['name','distance']].head(5)

,name,distance
1,The Grizzled,0.168988
2,The Lord of the Rings: The Card Game,0.176121
3,Keep the Heroes Out!,0.181035
4,Legends of Andor,0.189780
5,Shadows of Brimstone: Swamps of Death,0.190390


### About and Keywords

In [10]:
# collection_name = "about_and_keywords"

# about_and_key_df = df.copy().drop(columns=['description'])

# weaviate_client.create_bgg_collection(collection_name=collection_name, reset=False, use_about=True, attributes=all_columns_to_produce)

# about_df = weaviate_client.add_bgg_collection_batch(df=about_and_key_df, collection_name=collection_name, use_about=True, attributes=all_columns_to_produce)

# about_and_key_df.to_pickle('about_and_key_df.pkl')

In [11]:
collection_name = "about_and_keywords"

about_and_key_df = pd.read_pickle('about_and_key_df.pkl')
compare_df_storage[collection_name] = about_and_key_df

uuid = about_and_key_df.loc[about_and_key_df['name']==game_name]['UUID'].values[0]

similars = weaviate_client.find_near_objects(collection_name=collection_name, uuid=uuid, limit=50)

picks = {}

for item in similars:
    picks[str(item.uuid)]=item.metadata.distance
    
picks = pd.DataFrame.from_dict(picks, columns=['distance'], orient='index').sort_values(by='distance', ascending=True).reset_index().rename(columns={'index':'UUID'})

picks = picks.merge(about_and_key_df, on='UUID', how='inner')[1:]
picks[['name','distance']].head(5)

,name,distance
1,Frosthaven,0.157304
2,Gloomhaven: Jaws of the Lion,0.172408
3,Shadows of Brimstone: Swamps of Death,0.187645
4,The 7th Citadel,0.195305
5,Descent: Legends of the Dark,0.200807


### With Keywords, About, and Desc

In [12]:
# collection_name = "all_attributes"

# Optional scaling of attributes
# scaler = MinMaxScaler(feature_range=(0,1))
# for col in all_columns_to_produce:
#     df[col] = scaler.fit_transform(df[[col]])

# weaviate_client.create_bgg_collection(collection_name=collection_name, reset=False, use_about=True, use_description=True, attributes=all_columns_to_produce)

# all_attributes_df = weaviate_client.add_bgg_collection_batch(df=df, collection_name=collection_name, use_about=True, use_description=True, attributes=all_columns_to_produce)

# all_attributes_df.to_pickle('all_attributes_df.pkl')

In [13]:
collection_name = "all_attributes"

all_attributes_df = pd.read_pickle('all_attributes_df.pkl')
compare_df_storage[collection_name] = all_attributes_df

uuid = all_attributes_df.loc[all_attributes_df['name']==game_name]['UUID'].values[0]

similars = weaviate_client.find_near_objects(collection_name=collection_name, uuid=uuid, limit=50)

picks = {}

for item in similars:
    picks[str(item.uuid)]=item.metadata.distance
    
picks = pd.DataFrame.from_dict(picks, columns=['distance'], orient='index').sort_values(by='distance', ascending=True).reset_index().rename(columns={'index':'UUID'})

picks = picks.merge(all_attributes_df, on='UUID', how='inner')[1:]
picks[['name','distance']].head(5)

,name,distance
1,The Grizzled,0.169670
2,The Lord of the Rings: The Card Game,0.177131
3,Keep the Heroes Out!,0.184025
4,Shadows of Brimstone: Swamps of Death,0.189123
5,Legends of Andor,0.191765


### All Comparisons

In [14]:
compare_df_storage.keys()

dict_keys(['keywords_only', 'about_only', 'about_and_desc', 'about_and_keywords', 'all_attributes'])

In [15]:
test_games

['Dominion',
 'Gloomhaven',
 'Pandemic',
 'Splendor',
 'Viticulture Essential Edition',
 'Great Western Trail',
 'Terraforming Mars',
 'Chess',
 'Azul',
 'Codenames',
 'Pandemic Legacy: Season 1']

In [16]:
game_name = test_games[1]

test_results = {}
test_results[game_name] = {}

for collection_name, model_df in compare_df_storage.items():
    print(f"Model {collection_name}")

    uuid = model_df.loc[model_df['name']==game_name]['UUID'].values[0]

    similars = weaviate_client.find_near_objects(collection_name=collection_name, uuid=uuid, limit=50)

    picks = {}

    for item in similars:
        picks[str(item.uuid)]=item.metadata.distance
        
    picks = pd.DataFrame.from_dict(picks, columns=['distance'], orient='index').sort_values(by='distance', ascending=True).reset_index().rename(columns={'index':'UUID'})

    picks = picks.merge(model_df, on='UUID', how='inner')[1:]

    test_results[game_name][collection_name] = {x:y for x,y in zip(picks['name'], picks['distance'])}

Model keywords_only
Model about_only
Model about_and_desc
Model about_and_keywords
Model all_attributes


In [17]:
this_game_results = test_results[game_name]
this_game_results

{'keywords_only': {'Frosthaven': 0.25048398971557617,
  'Twilight Struggle': 0.3401280641555786,
  'Twilight Inscription': 0.3538649082183838,
  'Gloomhaven: Jaws of the Lion': 0.358690083026886,
  'Cloudspire': 0.3629630208015442,
  'Cry Havoc': 0.37528979778289795,
  'Verdant': 0.3771243691444397,
  'Mystic Vale': 0.37749582529067993,
  'Abyss': 0.38269877433776855,
  'Nemesis': 0.3827403783798218,
  'Meadow': 0.3844003677368164,
  'Welcome To...': 0.3845769166946411,
  'Thunderstone Quest': 0.3846357464790344,
  'Splendor': 0.3886149525642395,
  'Faraway': 0.3886687159538269,
  'Patchwork': 0.3900679349899292,
  'Twilight Imperium: Third Edition': 0.39512360095977783,
  '7 Wonders (Second Edition)': 0.39539647102355957,
  'Dungeon Lords': 0.39810657501220703,
  'London (Second Edition)': 0.39848560094833374,
  'Massive Darkness 2: Hellscape': 0.3986920118331909,
  'Boonlake': 0.39904558658599854,
  'Unmatched: Cobble & Fog': 0.3991039991378784,
  'Twilight Imperium: Fourth Edition':

In [18]:
results_df = pd.DataFrame(this_game_results)
results_df

,keywords_only,about_only,about_and_desc,about_and_keywords,all_attributes
Frosthaven,0.250484,0.159823,NaN,0.157304,NaN
Twilight Struggle,0.340128,NaN,NaN,NaN,NaN
Twilight Inscription,0.353865,NaN,NaN,NaN,NaN
Gloomhaven: Jaws of the Lion,0.358690,0.173312,NaN,0.172408,NaN
Cloudspire,0.362963,NaN,NaN,NaN,NaN
...,...,...,...,...,...
Lord of the Rings: The Confrontation,NaN,NaN,0.250244,NaN,NaN
Discworld: Ankh-Morpork,NaN,NaN,0.254385,NaN,NaN
Pandemic Legacy: Season 1,NaN,NaN,0.254801,NaN,NaN
Tainted Grail: The Fall of Avalon,NaN,NaN,0.256504,NaN,NaN


In [19]:
scaler = MinMaxScaler(feature_range=(0,1))
for col in results_df.columns:
    results_df[col] = scaler.fit_transform(results_df[[col]])
results_df

,keywords_only,about_only,about_and_desc,about_and_keywords,all_attributes
Frosthaven,0.000000,0.000000,NaN,0.000000,NaN
Twilight Struggle,0.511340,NaN,NaN,NaN,NaN
Twilight Inscription,0.589696,NaN,NaN,NaN,NaN
Gloomhaven: Jaws of the Lion,0.617219,0.132744,NaN,0.152334,NaN
Cloudspire,0.641593,NaN,NaN,NaN,NaN
...,...,...,...,...,...
Lord of the Rings: The Confrontation,NaN,NaN,0.928476,NaN,NaN
Discworld: Ankh-Morpork,NaN,NaN,0.975796,NaN,NaN
Pandemic Legacy: Season 1,NaN,NaN,0.980545,NaN,NaN
Tainted Grail: The Fall of Avalon,NaN,NaN,1.000000,NaN,NaN


In [22]:
results_df['average_score'] = results_df.mean(axis=1)
results_df.sort_values("average_score").head(20)

,keywords_only,about_only,about_and_desc,about_and_keywords,all_attributes,average_score
Frosthaven,0.000000,0.000000,NaN,0.000000,NaN,0.000000
Shadows of Brimstone: Swamps of Death,NaN,0.276723,0.244550,0.306008,0.281781,0.277265
Gloomhaven: Jaws of the Lion,0.617219,0.132744,NaN,0.152334,NaN,0.300766
The Lord of the Rings: The Card Game,NaN,0.545164,0.081511,0.583701,0.108080,0.329614
Legends of Andor,NaN,0.514400,0.237583,0.496585,0.320038,0.392152
Dungeons & Dragons: The Legend of Drizzt Board Game,NaN,NaN,0.351692,NaN,0.441756,0.396724
The 7th Citadel,NaN,0.371223,0.393861,0.383263,0.446738,0.398771
Descent: Legends of the Dark,NaN,0.385968,NaN,0.438756,NaN,0.412362
The Grizzled,NaN,0.848481,0.000000,0.851881,0.000000,0.425090
Warhammer Quest: The Adventure Card Game,NaN,0.454180,NaN,0.473134,NaN,0.463657


In [21]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

### Supplementary Data

In [ ]:
name_df = pd.read_pickle(f"{read_dir}games_clean.pkl")
name_df = name_df.sort_values("BayesAvgRating", ascending=False)[
            0 : 1000
        ].reset_index(drop=True)
name_df.head()

In [ ]:
mech_df = pd.read_pickle(f"{read_dir}mechanics_clean.pkl")
mech_df = explode_columnar_df(mech_df)
mech_df.head()

In [ ]:
subcats_df = pd.read_pickle(f"{read_dir}subcategories_clean.pkl")
subcats_df = explode_columnar_df(subcats_df)
subcats_df.head()

In [ ]:
themes_df = pd.read_pickle(f"{read_dir}themes_clean.pkl")
subcats_df = explode_columnar_df(themes_df)
subcats_df.head()